# Equation solving with JuMP
## Simple Reaction Equilibrium

We can use JuMP to solve nonlinear equations with physical constraints (e.g. positive mass)

$$
CO + 2H_2 => CH_3OH 
$$

The condition for chemical equilibrium of gases is given by:
$$
\begin{align}
Keq = & \frac{p_{CH_3OH}}{p_{CO} \ p_{H_2}^2}\\
= & \frac{y_{CH_3OH}}{y_{CO} \ y_{H_2}^2 P_{total}^2}
\end{align}
$$

We can write this condition as a set of equations in terms of molar flow rates to solve for the extent of reaction at equilibrium.

Assuming we input 100 kmol/hr of CO and 600 kmol/hr of H2 into our reactor, we get:

$$
\begin{align}
Keq = & \ \ \mu_{out,total}^2\frac{\mu_{out,CH_3OH}}{\mu_{out,CO} \ \mu_{out,H_2}^2}\\
& \mu_{in,k} \ge 0 \ \ \forall k \in \{CO,H_2,CH_3OH\}\\
& \mu_{out,k} \ge 0 \ \ \forall k \in \{CO,H_2,CH_3OH\}\\
& \mu_{out,total} \ge 0 \\
& \mu_{out,CO} = μ_{in,CO} - 100\xi \\
& \mu_{out,H_2} = μ_{in,H_2} - 2*100\xi \\
& \mu_{out,CH_3OH} = 100\xi \\
& \mu_{out,total} = \sum_{k \in \{CO,H_2,CH_3OH\}} \mu_{out,k}\\
& 0 \le \xi \le 1
\end{align}
$$


In [12]:
# call libraries to be used
using JuMP
using Ipopt

# define algebraic model
m = Model(solver=IpoptSolver());

In [13]:
# define sets and variables
K = ["CO","H2","CH3OH"]; # set of components

# define variables
@variable(m, μ_in[K]>=0)    # inlet molar flows (kmol/hr)
@variable(m, μ_out[K]>=0)   # outlet molar flows (kmol/hr)
@variable(m, μ_out_tot>=0)  # total outlet flow
@variable(m,0<=ξ<=1)        # extent of reactor [-]
@variable(m,P==150);         # reactor pressure [bar]


In [14]:
# define reaction data
T=300+273.15
Keq=10^(-12.275+4938/T) # reaction equilibrium @ 300 degC 

# Gibbs equilibrium condition
@constraint(m, μ_in["CO"] == 100)
@constraint(m, μ_in["H2"] == 600)
@constraint(m, μ_in["CH3OH"]== 0)
@constraint(m, μ_out["CO"]==μ_in["CO"]-100*ξ)
@constraint(m, μ_out["H2"]==μ_in["H2"]-2*100*ξ)
@constraint(m, μ_out["CH3OH"]==100*ξ)
@constraint(m, μ_out_tot==sum(μ_out[k] for k in K))

#Equilibrium condition
@NLconstraint(m, 
(μ_out["CH3OH"]/μ_out_tot)/(μ_out["CO"]*μ_out["H2"]*μ_out["H2"]/(μ_out_tot*μ_out_tot*μ_out_tot))== (P^2)*Keq);

In [15]:
solve(m)

This is Ipopt version trunk, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:       19
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        8
                     variables with only lower bounds:        7
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        8
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 6.00e+02 1.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
   1 

:Optimal

## Look at the resulting extent and moles

In [16]:
println("reaction extent = ",getvalue(ξ))
println("kmols/hr of CH3OH produced = ",getvalue(μ_out["CH3OH"]), "kmols/hr")

reaction extent = 0.7669148048452934
kmols/hr of CH3OH produced = 76.69148048452935kmols/hr
